## Global scRNAseq and functional data curation


In [1]:
# main output file with all samples merged
merged_adata_file_name = "all_merged.h5ad"

In [2]:
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.packages import importr
import os
import pandas as pd
import numpy as np
import anndata as ad
import hdf5plugin
import path_config

import warnings
warnings.filterwarnings("ignore")

Define paths to original data.<br>
Original data are .rds files with results of zUMIs. umicount exon all

In [3]:
received_data1_path = os.path.join(path_config.data_path,"SS3_22_291.dgecounts.rds")
received_data2_path = os.path.join(path_config.data_path,"SS3_23_049.dgecounts.rds") 
received_data3_path = os.path.join(path_config.data_path,"SS3_23_127.dgecounts.rds") 
received_data4_path = os.path.join(path_config.data_path,"SS3_23_193.dgecounts.rds")
received_data5_path = os.path.join(path_config.data_path,"SS3_23_195.dgecounts.rds")
received_data6_path = os.path.join(path_config.data_path,"SS3_23_325.dgecounts.rds")
received_data7_path = os.path.join(path_config.data_path,"SS3_23_327.dgecounts.rds")
received_data8_path = os.path.join(path_config.data_path,"P30855_101.dgecounts.rds")
received_data9_path = os.path.join(path_config.data_path,"P30855_103.dgecounts.rds")
received_data10_path = os.path.join(path_config.data_path,"P30855_105.dgecounts.rds")
received_data11_path = os.path.join(path_config.data_path,"P31406_101.dgecounts.rds")

# create a list of all original paths to automate reading
received_data_paths_list = [received_data1_path,received_data2_path,
                      received_data3_path,received_data4_path,
                      received_data5_path,received_data6_path,
                      received_data7_path,received_data8_path,
                      received_data9_path,received_data10_path,
                      received_data11_path
                      ]


Define paths to barcodes for each sample.<br>
These are the files from scilifeLab<br>
bc_set + tab + well_id

In [4]:
# txt file with well barcodes. the text contains the cell barcode corresponding to each position in the plate
well_barcode_sample1 = os.path.join(path_config.data_path, "SS3_22_291.well_barcodes.txt")
well_barcode_sample2 = os.path.join(path_config.data_path, "SS3_23_049.well_barcodes.txt")
well_barcode_sample3 = os.path.join(path_config.data_path, "SS3_23_127.well_barcodes.txt")
well_barcode_sample4 = os.path.join(path_config.data_path, "SS3_23_193.well_barcodes.txt")
well_barcode_sample5 = os.path.join(path_config.data_path, "SS3_23_195.well_barcodes.txt")
well_barcode_sample6 = os.path.join(path_config.data_path, "SS3_23_325.well_barcodes.txt")
well_barcode_sample7 = os.path.join(path_config.data_path, "SS3_23_327.well_barcodes.txt")
well_barcode_sample8 = os.path.join(path_config.data_path, "P30855_101.well_barcodes.txt")
well_barcode_sample9 = os.path.join(path_config.data_path, "P30855_103.well_barcodes.txt")
well_barcode_sample10 = os.path.join(path_config.data_path, "P30855_105.well_barcodes.txt")
well_barcode_sample11 = os.path.join(path_config.data_path, "P31406_101.well_barcodes.txt")

# create a list of barcodes per sample
all_well_barcodes_per_sample = [well_barcode_sample1,well_barcode_sample2,
                                well_barcode_sample3,well_barcode_sample4,
                                well_barcode_sample5,well_barcode_sample6,
                                well_barcode_sample7,well_barcode_sample8,
                                well_barcode_sample9,well_barcode_sample10,
                                well_barcode_sample11
                                ]


Define paths to files with functional experimental data

In [5]:

# upload excel with red and green flourecence from campari (has information about position in plate and fluorecence level)
florecence_1_file_path = os.path.join(path_config.data_path, "well_1_data_base_labeled.csv")
florecence_2_file_path = os.path.join(path_config.data_path, "well_2_data_base_labeled.csv")
florecence_3_file_path = os.path.join(path_config.data_path, "well_3_data_base_labeled.csv")
florecence_4_file_path = os.path.join(path_config.data_path, "well_4_data_base_labeled.csv")
florecence_5_file_path = os.path.join(path_config.data_path, "well_5_data_base_labeled.csv")
#######################################################################################################################################
###BE CAREFULL WITH PROJECT NAME AND PLATE NUMBER ASSIGNMENT!
### WE SWAP WERE WHAT IS 6 AND 7 IN THE STIMULI FILE SO BARCODES ARE WITH THEIR CORRECT STIMULI ID
#######################################################################################################################################
florecence_6_file_path = os.path.join(path_config.data_path, "well_7_data_base_labeled.csv") # swapped
florecence_7_file_path = os.path.join(path_config.data_path, "well_6_data_base_labeled.csv") # swapped
florecence_8_file_path = os.path.join(path_config.data_path, "well_8_data_base_labeled.csv")
florecence_9_file_path = os.path.join(path_config.data_path, "well_9_data_base_labeled.csv")
florecence_10_file_path = os.path.join(path_config.data_path, "well_10_data_base_labeled.csv")
florecence_11_file_path = os.path.join(path_config.data_path, "well_11_data_base_labeled.csv")

# create a list of files with metadata from the lab
all_fluorescence_paths = [florecence_1_file_path,florecence_2_file_path,
                        florecence_3_file_path,florecence_4_file_path,
                        florecence_5_file_path,florecence_6_file_path,
                        florecence_7_file_path,florecence_8_file_path,
                        florecence_9_file_path,florecence_10_file_path,
                        florecence_11_file_path
                        ]

# Read and organize UMI counts

In [6]:
# Create a list of all data frames. One data frame per sample
all_samples_dfs = []
# check how many cells were in each of the data sets
all_total_barcodes = []
for i,raw_R_data in enumerate(received_data_paths_list):
    print(f"Processing sample",i+1)
    #print(received_data_paths_list[i])
    # Read the RDS file
    r_readRDS = robjects.r['readRDS']
    received_data = r_readRDS(raw_R_data)
    # Extract "umicount" data
    new_data = received_data.rx2("umicount")
    exon = new_data.rx2("exon").rx2("all")
    # Convert dgCMatrix to a dense matrix in R
    as_matrix = robjects.r['as.matrix']
    dense_matrix = as_matrix(exon)  # Convert to dense matrix
    # Convert dense matrix to NumPy array
    exon_matrix = np.array(dense_matrix)
    
    # Extract row and column names
    base = importr('base')
    rownames = list(base.rownames(exon))
    colnames = list(base.colnames(exon))

    # Create a pandas DataFrame
    my_df = pd.DataFrame(exon_matrix, index=rownames, columns=colnames)

    # check how many cells were in each of the data sets
    total_barcodes = my_df.shape[1]
    all_total_barcodes.append(total_barcodes)
    # they are different sizes
    all_samples_dfs.append(my_df)
    print("genes, cells")
    print(my_df.shape)
    print()

Processing sample 1


R[write to console]: Loading required package: Matrix



genes, cells
(27978, 384)

Processing sample 2
genes, cells
(29575, 384)

Processing sample 3
genes, cells
(28424, 384)

Processing sample 4
genes, cells
(29285, 384)

Processing sample 5
genes, cells
(28390, 384)

Processing sample 6
genes, cells
(28190, 384)

Processing sample 7
genes, cells
(28706, 384)

Processing sample 8
genes, cells
(29363, 384)

Processing sample 9
genes, cells
(29682, 384)

Processing sample 10
genes, cells
(28587, 384)

Processing sample 11
genes, cells
(28914, 384)



In [7]:
# merge whole dataframe, so that all will have the same amount of genes. If some did not contain certain gene, count=0
unified_df = pd.concat(all_samples_dfs,axis=1)
# replace all NaN values with zero counts
unified_df = unified_df.fillna(0.0)
unified_df

,AACCATCGGCAACTACCACT,AACCATCGGCAAGTTATCGG,AACCATCGGCCACAATCCAC,AACCATCGGCCACTAACCGG,AACCATCGGCCCGTTCGTAT,AACCATCGGCCGACGGTTGT,AACCATCGGCCGAGTCATTA,AACCATCGGCGAAGCTAGCC,AACCATCGGCGTAACGTCTA,AACCATCGGCTCCAATGCAA,...,TTGTCTGCAAGCACAGTATT,TTGTCTGCAATACAACTAGC,TTGTCTGCAATAGGTGGCGA,TTGTCTGCAATAGTTAGGCT,TTGTCTGCAATCTGCAACAA,TTGTCTGCAATGCATCATAC,TTGTCTGCAATGGCCTTAAT,TTGTCTGCAATTACAACACG,TTGTCTGCAATTCAGGAGAT,TTGTCTGCAATTCTCTGATC
0610005C13Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0610009E02Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0610009L18Rik,3.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0
0610010K14Rik,2.0,2.0,4.0,0.0,4.0,3.0,1.0,0.0,3.0,3.0,...,2.0,4.0,2.0,8.0,10.0,5.0,1.0,6.0,5.0,3.0
0610025J13Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trav6-7-dv9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Usp9y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vmn1r80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wfdc11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# decompose from unified
all_decomposed_samples = []
for i, total_barcodes in enumerate(all_total_barcodes):
    if i == 0: # for the first one
        sample_df = unified_df.iloc[:,0:all_total_barcodes[i]]
        till_col = all_total_barcodes[i]+all_total_barcodes[i+1]
    elif i == 1:
        sample_df = unified_df.iloc[:,all_total_barcodes[0]:till_col]
        from_col = till_col
        till_col = from_col+all_total_barcodes[i+1]
    elif i == len(all_total_barcodes)-1:
        sample_df = unified_df.iloc[:,from_col:till_col]
    else:
        sample_df = unified_df.iloc[:,from_col:till_col]
        from_col = till_col
        till_col = from_col+all_total_barcodes[i+1]
    #print(sample_df.shape[1]) # number of cells
    all_decomposed_samples.append(sample_df)

Create single adata objects

In [9]:
# Convert all df to AnnData objects 
# export to h5ad files
all_adatas = []
for i, df in enumerate(all_decomposed_samples):
    # transpose before creating anndata object, so that gene ids are columns and barcodes are rows
    transposed_rds_sample= all_decomposed_samples[i].transpose()# Create an AnnData object
    adata = ad.AnnData(
        X=transposed_rds_sample.values,  # Your data as numpy array
        obs={'barcode': transposed_rds_sample.index},
        var={'gene_id': transposed_rds_sample.columns}  
    )
    # add sample metadata
    sample = "Sample_" + str(i+1)
    adata.obs['sample'] = sample

    all_adatas.append(adata)

## Merge functional experimental data with scRNAseq data

Helper function for adding functional tags to scRNAseq data 

In [10]:
# returns ordered well ids, stimulus and fluorescence from each sample
def barcode_check(barcode_list,mergedRes):
    
    well_id = ["NA" for el in barcode_list] # these are strings (i.e "A1")
    red_f = np.zeros((len(barcode_list),))
    green_f = np.zeros((len(barcode_list),))
    stimulus = np.zeros((len(barcode_list),))
    plate_number = np.zeros((len(barcode_list),))
    area = np.zeros((len(barcode_list),))
    cell_id = np.zeros((len(barcode_list),))


    for i in range(len(barcode_list)):
        x = barcode_list[i]
        index = barcode_list.index(x)
        if len(np.where(np.isnan(mergedRes['stimulus'].where(mergedRes['bc_set'] == x))==False)):
            if np.where(mergedRes['bc_set'] == x)[0]:
                position = np.where(np.isnan(mergedRes['stimulus'].where(mergedRes['bc_set'] == x))==False)[0][0]
                stimulus[index] = mergedRes['stimulus'].where(mergedRes['bc_set'] == x)[position]
                red_f[index] = mergedRes['red_f'].where(mergedRes['bc_set'] == x)[position]
                green_f[index] = mergedRes['green_f'].where(mergedRes['bc_set'] == x)[position]
                well_id[index] = mergedRes['well_id'].where(mergedRes['bc_set'] == x)[position]
                plate_number[index] = mergedRes['well_number'].where(mergedRes['bc_set'] == x)[position]
                area[index] = mergedRes['AREA'].where(mergedRes['bc_set'] == x)[position]
                cell_id[index] = mergedRes['cell_number'].where(mergedRes['bc_set'] == x)[position]
                
    # replace all nans to zeros
    np.nan_to_num(well_id,copy=False)
    np.nan_to_num(stimulus,copy=False)
    np.nan_to_num(red_f,copy=False)
    np.nan_to_num(green_f,copy=False)
    np.nan_to_num(plate_number,copy=False)
    np.nan_to_num(area,copy=False)
    np.nan_to_num(cell_id,copy=False)

    return well_id,stimulus,red_f,green_f,plate_number,area, cell_id

 
# Merge all into one

In [11]:
# collect all infomration per sample into single adata objects
all_complete_adatas = []
# in the end we will have merged_adata
merged_adata = None
for i, sample in enumerate(all_adatas):
    current_adata = all_adatas[i]
    flouresence_sample = pd.read_csv(all_fluorescence_paths[i])
    well_df_sample = pd.read_csv(all_well_barcodes_per_sample[i], delimiter = "\t")
    # read barcodes in the same order as in the anndata object
    barcodes = [el[0] for el in list(current_adata.obs.values)]
    # merge data base. Merge information about cell barcode and fluorecence 
    #(using as common demonimator the position in the plate)
    mergedRes = pd.merge(well_df_sample , flouresence_sample, on ='well_id')
    well_id,stimulus,red_f,green_f,plate_number,area, cell_id = barcode_check(barcodes,mergedRes)
    current_adata.var['gene_name'] = current_adata.var['gene_id']
    current_adata.var.index = current_adata.var["gene_name"]
    ### add campari information to adata structure
    current_adata.obs['stimulus'] = stimulus
    current_adata.obs['red'] = red_f
    current_adata.obs['green'] = green_f
    current_adata.obs['well_id'] = well_id
    current_adata.obs['plate_number'] = plate_number
    current_adata.obs['area'] = area
    current_adata.obs['cell_id'] = cell_id

    # add to list of all adata objects
    all_complete_adatas.append(current_adata)

    # start concatenating when at least 2 adata objects were in all_complete_adatas
    if i > 0 and i < (len(all_adatas)):
        if i == 1:
            sample_1 = "sample1"
            sample_2 = "sample2"
            merged_adata = all_complete_adatas[0].concatenate(all_complete_adatas[i], join='outer', batch_categories=[sample_1, sample_2], index_unique='-')
        else:
            sample_1 = sample_1+str(i)
            sample_2 = "sample"+str(i+1)
            merged_adata = merged_adata.concatenate(all_complete_adatas[i], join='outer', batch_categories=[sample_1, sample_2], index_unique='-')

In [12]:
# assign our merged data
adata = merged_adata.copy()

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
print('Number of cells:', adata.X.shape[0]) # len(adata.obs_names)
print('Number of genes:', adata.X.shape[1]) # len(adata.var_names)

Number of cells: 4224
Number of genes: 35546


In [13]:
adata

AnnData object with n_obs × n_vars = 4224 × 35546
    obs: 'barcode', 'sample', 'stimulus', 'red', 'green', 'well_id', 'plate_number', 'area', 'cell_id', 'batch'
    var: 'gene_id', 'gene_name'

## Export merged anndata object

In [15]:
# export merged data
adata.write_h5ad(
    os.path.join(path_config.output_path,merged_adata_file_name)
)